# ISTAC playground

**NOTE: You must select the istac kernel to run this playbook!**

This playbook uses a virtualenv that must be setup beforehand. This is a one-time only process, recorded here for future reference.
The steps to create a virtualenv for this project and make it available as a jupyter kernel are:

```bash
# Move to the proper folder
cd /home/jovyan/work/istac
# Install pipenv
pip install pipenv
# Create the venv and install dependencies
pipenv install
# Activate the shell
pipenv shell
# Create a jupyter core
python -m ipykernel install --user --name=istac
```

Now you can select the "istac" core when running this playbook.

In [50]:
# Import istac lib into your app
import istac

In [51]:
# Collect all indicators
import aiohttp

async with aiohttp.ClientSession() as session:
    indicators = [ind async for ind in istac.indicators(session)]

In [52]:
# Now you can list the indicators, e.g.
from pprint import pprint

pprint([ind.code for ind in indicators[:10]])

['AFILIACIONES',
 'TURISTAS',
 'EMPLEO_REGISTRADO_AGRICULTURA',
 'EMPLEO_REGISTRADO_HOSTELERIA',
 'EMPLEO_REGISTRADO_INDUSTRIA',
 'EMPLEO_REGISTRADO_SERVICIOS',
 'POBLACION_INACTIVA',
 'POBLACION_INACTIVA_HOMBRES',
 'POBLACION_INACTIVA_MUJERES',
 'PARO_REGISTRADO']


In [53]:
# And get data for same indicator, e.g.
async with aiohttp.ClientSession() as session:
    df = await istac.indicator_data(session, 'TURISTAS', {
        'granularity': 'TIME[MONTHLY]',
        'representation': 'MEASURE[ABSOLUTE]',
        'fields': '-observationsMetadata'
    })

In [54]:
# further manipulate the data with pandas' DataFrame tools
pprint(df.head(10).drop('_meta', axis=1).to_csv())

('_offset,F,GEOGRAPHICAL,TIME,MEASURE\n'
 '0,1325924,ES70,2019M12,ABSOLUTE\n'
 '1,1289081,ES70,2019M11,ABSOLUTE\n'
 '2,1291653,ES70,2019M10,ABSOLUTE\n'
 '3,1128384,ES70,2019M09,ABSOLUTE\n'
 '4,1289910,ES70,2019M08,ABSOLUTE\n'
 '5,1272383,ES70,2019M07,ABSOLUTE\n'
 '6,1127367,ES70,2019M06,ABSOLUTE\n'
 '7,1066023,ES70,2019M05,ABSOLUTE\n'
 '8,1240146,ES70,2019M04,ABSOLUTE\n'
 '9,1477483,ES70,2019M03,ABSOLUTE\n')
